### Testes básicos do funcionamento do CCScore

In [10]:
import sys
sys.path.append('../ccscore/')
from infernal import feature_extraction as fe
from infernal import datastructures as ds
import spacy

In [11]:
nlp = spacy.load('pt_core_news_lg')

In [12]:
texto = '''Defendida por agentes do mercado financeiro e uma das bandeiras da equipe econômica do governo Jair Bolsonaro, o projeto de autonomia do Banco Central (BC) deve avançar na Câmara só após a reforma tributária andar, no que depender do presidente da Casa, Rodrigo Maia (DEM-RJ). Para ele, o projeto sobre a instituição presidida por Roberto Campos Neto não é urgente no curto prazo.
"Aceito votar autonomia do Banco, aceito, é claro, votar os depósitos voluntários, mas aí temos que organizar melhor a pauta até o fim do ano. É só o governo ter boa vontade na reforma tributária", disse Maia, ao participar de evento organizado pelo Itaú. "A reforma tributária tem importância muito maior que autonomia do Banco Central", comentou.
'''


texto

'Defendida por agentes do mercado financeiro e uma das bandeiras da equipe econômica do governo Jair Bolsonaro, o projeto de autonomia do Banco Central (BC) deve avançar na Câmara só após a reforma tributária andar, no que depender do presidente da Casa, Rodrigo Maia (DEM-RJ). Para ele, o projeto sobre a instituição presidida por Roberto Campos Neto não é urgente no curto prazo.\n"Aceito votar autonomia do Banco, aceito, é claro, votar os depósitos voluntários, mas aí temos que organizar melhor a pauta até o fim do ano. É só o governo ter boa vontade na reforma tributária", disse Maia, ao participar de evento organizado pelo Itaú. "A reforma tributária tem importância muito maior que autonomia do Banco Central", comentou.\n'

Processamento básico dos textos é feito com Spacy, 
mas de forma interno às classes do ccscore

In [13]:
textdoc = nlp(texto)

In [14]:
from itertools import tee

def pairwise(iterable):
    '''
    Receive an iterable and return
    an list with tuples of this elements
    is this way    
    s -> (s0,s1), (s1,s2), (s2, s3), ...
    '''
    a, b = tee(iterable)    
    next(b, None)  # next(b) it's Ok 
    return zip(a, b)


Listando:    
<ol>
    <li>Foco Explícito (FE)</li>
    <li>Lista intermediária do FE</li>    
    <li>Lista das Entidades Nomeadas</li>
    <li>Menções da DBPedia</li>
</ol>

### Carga e alinhamento dos dados anotados pelo PALAVRAS

In [19]:
import confapp as config
config.TEP2_PATH = "../ccscore/data/tep2.pickle"
import helper_palavras as h_pal 
from text_document import TextDocument


ARQ_TEXTO = "../ccscore/data/texto_exemplo.txt"
ARQ_PALAVRAS = "../ccscore/data/texto_exemplo_anotado.html"
ARQ_PALAVRAS_ORIG = "../ccscore/data/texto_exemplo.txt"

try:
    with open(ARQ_TEXTO, 'r') as f_text:
            orig_text = " ".join(f_text.readlines())
except IOError:
    print(f"Erro ao tentar abrir o arquivo {ARQ_TEXTO}")

tp = h_pal.parse_file_toclass(ARQ_PALAVRAS,
                              ARQ_PALAVRAS_ORIG)
                              
td = TextDocument(orig_text, tp)

In [20]:
pares = list(pairwise(td.sentences))

In [22]:
# Teste do extrator de features do pacote infernal
extrator = fe.FeatureExtractor(both=True)
extrator.get_feature_names()

['Bleu T->H',
 'Bleu H->T',
 'Cosine TED',
 'Cosine TED / Length T',
 'Cosine TED / Length H',
 'Depedency Overlap T',
 'Depedency Overlap H',
 'Nominalization in H',
 'Nominalization in T',
 'Length Ratio',
 'Verb Structure T->H',
 'Verb Structure H->T',
 'Negated Verb',
 'Quantity Match',
 'Quantity Mismatch',
 'Sentence Cosine',
 'Simple TED',
 'TED / Length T',
 'TED / Length H',
 'Soft Overlap T->H',
 'Soft Overlap H->T',
 'Overlap T',
 'Overlap H',
 'Synonym Overlap T',
 'Synonym Overlap H',
 'Non-aligned Entity T',
 'Non-aligned Entity H',
 'Has Aligned Entity']

In [23]:
from infernal.datastructures import Entailment 
s1,s2 = pares[0]

par = ds.Pair(s1,s2, 0, Entailment.none)

par

### Uso do dicionário Tep2

In [25]:
import helper_tools as htools
from tep2 import GrupoSinonimo

htools.dic_tep2.get_sinonimos("governo")

['estado',
 'controle',
 'domínio',
 'regência',
 'governança',
 'administração',
 'regime',
 'governação',
 'regimento',
 'regímen',
 'norteamento',
 'direção',
 'orientação',
 'norteio']

In [26]:
for i, sent in enumerate(td.sentences):
    print()
    print("ID: ", str(i))   
    print(sent.tokens_dbpedia_metions)
    print("Texto Original: ", sent.text)    


ID:  0
{'mercado financeiro': mercado financeiro, 'governo Jair Bolsonaro': governo Jair Bolsonaro, 'Banco Central': Banco Central, 'reforma tributária': reforma tributária, 'Rodrigo Maia': Rodrigo Maia}
Texto Original:  Defendida por agentes do mercado financeiro e uma das bandeiras da equipe econômica do governo Jair Bolsonaro, o projeto de autonomia do Banco Central (BC) deve avançar na Câmara só após a reforma tributária andar, no que depender do presidente da Casa, Rodrigo Maia (DEM-RJ).

ID:  1
{'Roberto Campos Neto': Roberto Campos Neto}
Texto Original:  Para ele, o projeto sobre a instituição presidida por Roberto Campos Neto não é urgente no curto prazo.

ID:  2
{}
Texto Original:  "Aceito votar autonomia do Banco, aceito, é claro, votar os depósitos voluntários, mas aí temos que organizar melhor a pauta até o fim do ano.

ID:  3
{'boa': boa, 'reforma tributária': reforma tributária, 'Itaú': Itaú}
Texto Original:  É só o governo ter boa vontade na reforma tributária", disse M

In [27]:
import helper_tools as htools

#htools.get_dbpedia_mentions(texto)
dbpedia_mentions = []
dbpedia_mentions_entries = htools.get_dbpedia_entries(texto)
for key, entry in dbpedia_mentions_entries.items():
    if entry and key == 'raw_text':        
        dbpedia_mentions.append(entry['raw_text'].replace("'",''))

#print(dbpedia_mentions)
#print(dbpedia_mentions_entries)

print("\n\n".join(["{}: {}".format(k,v) for k,v in dbpedia_mentions_entries.items()]))

mercado financeiro: {'URI': 'http://pt.dbpedia.org/resource/Mercado_financeiro', 'dbpedia_text': 'Mercado financeiro', 'proeminence': '250', 'dbpedia_types': [''], 'types': [''], 'raw_text': 'mercado financeiro', 'pos': 25}

governo Jair Bolsonaro: {'URI': 'http://pt.dbpedia.org/resource/Governo_Jair_Bolsonaro', 'dbpedia_text': 'Governo Jair Bolsonaro', 'proeminence': '269', 'dbpedia_types': [''], 'types': [''], 'raw_text': 'governo Jair Bolsonaro', 'pos': 87}

Banco Central: {'URI': 'http://pt.dbpedia.org/resource/Banco_Central_do_Brasil', 'dbpedia_text': 'Banco Central do Brasil', 'proeminence': '634', 'dbpedia_types': [''], 'types': [''], 'raw_text': 'Banco Central', 'pos': 704}

reforma tributária: {'URI': 'http://pt.dbpedia.org/resource/Reforma_tributária', 'dbpedia_text': 'Reforma tributária', 'proeminence': '33', 'dbpedia_types': [''], 'types': [''], 'raw_text': 'reforma tributária', 'pos': 640}

Rodrigo Maia: {'URI': 'http://pt.dbpedia.org/resource/Rodrigo_Maia', 'dbpedia_text'

In [28]:
import requests
from bs4 import BeautifulSoup
import re

page = requests.get("http://pt.dbpedia.org/resource/Pleno_emprego")
soup = BeautifulSoup(page.text, 'html.parser')

a_elements = soup.find_all('a', text=re.compile(".*Categoria.*"))

def get_categorias_dbpedia(resource):
    page = requests.get("http://pt.dbpedia.org/resource/{}".format(resource))
    soup = BeautifulSoup(page.text, 'html.parser')
    a_elements = soup.find_all('a', text=re.compile(".*Categoria.*"))
    
    categorias = set([])
    for elem in a_elements:
        texto = "".join(elem.text.split())
        texto = texto.split("dbr:Categoria:")[1]
        texto = texto.replace("_", " ")
        categorias.add(texto)

    return categorias
    


In [29]:
categorias = set([])
for elem in a_elements:
    texto = "".join(elem.text.split())
    texto = texto.split("dbr:Categoria:")[1]
    texto = texto.replace("_", " ")
    categorias.add(texto)
    
categorias

{'Macroeconomia', 'Terminologia econômica'}

In [30]:
get_categorias_dbpedia("Reforma_tributária")

{'Direito tributário', 'Macroeconomia', 'Política', 'Sociedade'}

In [ ]:
#print(list(dbpedia_mentions_entries.values())[5])
print(htools.get_dbpedia_entries(texto, formated=False))


In [31]:
# Recuperar os tokens que constituem os spans relacionados
# as menções obtidas da dbpedia spotlight

ent1 = list(dbpedia_mentions_entries.values())[0]

tk0 = None
for t in td.sentences[0].annotated:
    if  t.idx == int(ent1['pos']):
        tk0 = t


sp1 = td.sentences[0].annotated[tk0.i:tk0.i+len(ent1['raw_text'].split())]
for t in sp1:
    print(type(t))
    print(t.pos_)
    print(t.text)

<class 'spacy.tokens.token.Token'>
NOUN
mercado
<class 'spacy.tokens.token.Token'>
ADJ
financeiro


In [58]:
import pickle

tep2 = None
with open(config.TEP2_PATH, 'rb') as f:
    tep2 = pickle.load(f)



In [73]:
from infernal import openwordnetpt as own
own.load_wordnet("../ccscore/data/own-pt.pickle")


In [62]:
dir(tep2)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'add',
 'classes',
 'data',
 'get',
 'get_antonimos',
 'get_by_classe',
 'get_classe',
 'get_sinonimos',
 'idx_antonimos',
 'set_antonimos',
 'set_classes',
 'set_sinonimos',
 'sinonimos']

In [74]:
#syns = own.find_synonyms("governo")
#own.are_synonyms("governo", "instituição")

import helper_tools as htools
from infernal import openwordnetpt as own

for sent in td.sentences:
    print("="*100)
    print(f"Sentenca: {sent.text}")
    print("="*100, '\n')
    
    for fe_elem in sent.list_fe:
        print(fe_elem, ":")
        try:
            print(f"WordNet: {own.find_synonyms(htools.cogroo_lemmatize(fe_elem.lower()))}")
            print(f"TeP2: {tep2.get_sinonimos(htools.cogroo_lemmatize(fe_elem.lower()))}")
        except Exception as e:
            print(str(e))
        print("-------------------------")


Sentenca: Defendida por agentes do mercado financeiro e uma das bandeiras da equipe econômica do governo Jair Bolsonaro, o projeto de autonomia do Banco Central (BC) deve avançar na Câmara só após a reforma tributária andar, no que depender do presidente da Casa, Rodrigo Maia (DEM-RJ).

Rodrigo Maia :
WordNet: set()
TeP2: []
-------------------------
presidente :
WordNet: {'diretor', 'presidente', 'presidente dos Estados Unidos da América', 'presidenta', 'reitor'}
TeP2: ['presidência']
-------------------------
autonomia :
WordNet: {'liberalismo', 'autonomia', 'independência'}
TeP2: ['desprendimento', 'auto-suficiência', 'independência', 'emancipação', 'independência', 'liberdade']
-------------------------
que :
WordNet: set()
TeP2: ['quão']
-------------------------
agentes :
WordNet: {'realizador', 'guarda', 'agente', 'policial', 'agente teatral', 'olheiro', 'fazedor', 'trabalhador', 'polícia'}
TeP2: ['propulsor', 'impulsor', 'motor', 'fator', 'causa', 'delegado', 'polícia', 'invest

WordNet: {'Edição jornalística', 'redação', 'redacção'}
TeP2: ['composição']
-------------------------
governo :
WordNet: {'governo', 'governação', 'regime', 'administração'}
TeP2: ['estado', 'controle', 'domínio', 'regência', 'governança', 'administração', 'regime', 'governação', 'regimento', 'regímen', 'norteamento', 'direção', 'orientação', 'norteio']
-------------------------
atuação :
WordNet: {'representação', 'interpretação', 'atuação'}
TeP2: ['obra', 'feito', 'ação', 'ato', 'exercício', 'influxo', 'poder', 'influência', 'ação', 'jurisdição', 'influição', 'performance', 'desempenho']
-------------------------
proposta :
WordNet: {'proposição', 'proposta', 'sugestão'}
TeP2: ['alvitre', 'sugestâo', 'proposição']
-------------------------
escopo :
WordNet: {'escopo', 'áreas', 'alcance'}
TeP2: ['fim', 'intuito', 'motivo', 'ponto', 'alvo', 'mira', 'fito', 'fim']
-------------------------
Sentenca: Maia já reclamou outras vezes da falta de empenho e atuação do governo para se aprovar 

WordNet: set()
TeP2: ['quão']
-------------------------
exemplo :
WordNet: {'lição', 'ocasião', 'ocorrência', 'caso', 'exemplo', 'exemplar', 'modelo'}
TeP2: ['escola', 'ensinamento', 'lição', 'retrato', 'modelo', 'traslado', 'translado', 'lição']
-------------------------
Banco Central :
WordNet: {'Banco Central', 'banco central'}
TeP2: []
-------------------------
senadores :
WordNet: {'senadora', 'senador'}
TeP2: []
-------------------------
proposta :
WordNet: {'proposição', 'proposta', 'sugestão'}
TeP2: ['alvitre', 'sugestâo', 'proposição']
-------------------------
Sentenca: Sobre a reforma tributária, Maia deu sinais de que quer aprovar o projeto antes de deixar a presidência da Casa e acredita que com acordo pode fazer isso rapidamente.

sinais :
WordNet: {'indício', 'Simbolo', 'gesto', 'Sinalização telefônica', 'vestígio', 'símbolo', 'sinal', 'Símbolos', 'lunar', 'agouro', 'presságio', 'traço', 'aceno', 'entrada', 'emblema', 'sinal de trânsito', 'mancha', 'sombra', 'indicação',

In [66]:
for i, sent in enumerate(td.sentences):
    for text_token in sent.list_fe:    
        pos_token = sent.tokens_fe_pos_tags[text_token].pos_ if text_token in sent.tokens_fe_pos_tags.keys() else None
        print(text_token, pos_token)

Rodrigo Maia None
presidente NOUN
autonomia NOUN
que PRON
agentes NOUN
projeto NOUN
mercado financeiro None
Jair Bolsonaro None
Casa NOUN
bandeiras NOUN
Câmara NOUN
BC None
DEM-RJ None
equipe NOUN
reforma tributária None
governo Jair Bolsonaro None
Banco Central None
ele PRON
projeto NOUN
prazo NOUN
instituição NOUN
Roberto Campos Neto None
fim NOUN
depósitos NOUN
pauta NOUN
ano NOUN
Banco None
autonomia NOUN
Itaú None
reforma tributária None
governo NOUN
vontade NOUN
evento NOUN
boa None
Maia None
reforma tributária None
autonomia NOUN
que PRON
importância NOUN
Banco Central None
terça-feira NOUN
aval NOUN
Senado NOUN
BC None
autonomia NOUN
projeto NOUN
deputados NOUN
lei NOUN
principal NOUN
objetivo NOUN
controle NOUN
metas NOUN
pleno emprego None
preços NOUN
prejuízo NOUN
flutuações NOUN
atividade NOUN
País NOUN
nível NOUN
texto NOUN
BC None
redação NOUN
governo NOUN
atuação NOUN
proposta NOUN
escopo NOUN
se PRON
vezes NOUN
governo NOUN
falta NOUN
empenho NOUN
medida NOUN
Maia None


In [67]:
sent = td.sentences[0]

import itertools as itt

it_fe_pairs = itt.permutations(sent.list_fe, 2)


#print([elem_fe_pair for elem_fe_pair in it_fe_pairs])

In [68]:
import itertools as itt

it_fe_pairs = itt.permutations(sent.list_fe, 2)
head_elem = None
is_part_of_other = False
elems_to_exclude = set([])
for a,b in it_fe_pairs:    
    if len(a.split()) == 1:    
        if a in b:
            elems_to_exclude.add(a)
            is_part_of_other = True            
            
print(sent.list_fe,' - ', len(sent.list_fe))    
list_fe = list(set(sent.list_fe).difference(set(elems_to_exclude)))
print(list_fe,' - ', len(list_fe))    

['Rodrigo Maia', 'presidente', 'autonomia', 'que', 'agentes', 'projeto', 'mercado financeiro', 'Jair Bolsonaro', 'Casa', 'bandeiras', 'Câmara', 'BC', 'DEM-RJ', 'equipe', 'reforma tributária', 'governo Jair Bolsonaro', 'Banco Central']  -  17
['Rodrigo Maia', 'presidente', 'autonomia', 'que', 'agentes', 'projeto', 'mercado financeiro', 'Jair Bolsonaro', 'Casa', 'bandeiras', 'Câmara', 'BC', 'DEM-RJ', 'equipe', 'reforma tributária', 'governo Jair Bolsonaro', 'Banco Central']  -  17


In [69]:
ss = td.sentences[3]

#print(dir(ss.anotated))
#ss.anotated.start

sentence_tokens = [token for token in ss.annotated]
type(sentence_tokens[0])
sentence_tokens

[É,
 só,
 o,
 governo,
 ter,
 boa,
 vontade,
 na,
 reforma,
 tributária,
 ",
 ,,
 disse,
 Maia,
 ,,
 ao,
 participar,
 de,
 evento,
 organizado,
 pelo,
 Itaú,
 .]

In [82]:
import pickle

tep2 = None
with open('../ccscore/data/tep2.pickle', 'rb') as f:
    tep2 = pickle.load(f)
    
tep2.get_sinonimos('governo')

['estado',
 'controle',
 'domínio',
 'regência',
 'governança',
 'administração',
 'regime',
 'governação',
 'regimento',
 'regímen',
 'norteamento',
 'direção',
 'orientação',
 'norteio']

In [83]:
from cogroo_interface import Cogroo
cogroo = Cogroo.Instance()
doc = cogroo.analyze('bandeiras')
print(f"token pos: {doc.sentences[0].tokens}")
print(f"token lemma: {cogroo.lemmatize('bandeiras')}")

print(f"token lemma: {cogroo.lemmatize('abacelara-lhe')}")



token pos: [bandeiras#n F=P]
token lemma: bandeira
token lemma:  


In [84]:
FE_POS_TAGS = {"NOUN", "PRON", "PROPN"}
POS_PLURAL = {}
tk_exemplo = None
for sent in td.sentences:
    print(f"Sentenca: {sent.text}")
    for token in sent.annotated:
        #if token.text == "agentes":
                #print(token.text, " ", token.tag_)
                                
                if tk_exemplo is None:
                    tk_exemplo = token
                if token.pos_ in FE_POS_TAGS and \
                   "Number=Plur" in token.tag_ :# and token.tag_ in POS_PLURAL:
                    print(f"Texto: {token.text}")
                    print(f"Lemma: {token.lemma_}")
                    print(f"Morph: {token.text}")
                    print(f"POS: {token.pos_}")
                    print(f"Lemma CoGroo: {cogroo.lemmatize(token.text)}")
                    print("--"*30)


Sentenca: Defendida por agentes do mercado financeiro e uma das bandeiras da equipe econômica do governo Jair Bolsonaro, o projeto de autonomia do Banco Central (BC) deve avançar na Câmara só após a reforma tributária andar, no que depender do presidente da Casa, Rodrigo Maia (DEM-RJ).
Texto: agentes
Lemma: agente
Morph: agentes
POS: NOUN
Lemma CoGroo: agente
------------------------------------------------------------
Texto: bandeiras
Lemma: bandeiro
Morph: bandeiras
POS: NOUN
Lemma CoGroo: bandeira
------------------------------------------------------------
Sentenca: Para ele, o projeto sobre a instituição presidida por Roberto Campos Neto não é urgente no curto prazo.
Sentenca: "Aceito votar autonomia do Banco, aceito, é claro, votar os depósitos voluntários, mas aí temos que organizar melhor a pauta até o fim do ano.
Texto: depósitos
Lemma: depósito
Morph: depósitos
POS: NOUN
Lemma CoGroo: depósito
------------------------------------------------------------
Sentenca: É só o gover

In [85]:
print("\n".join([l for l in nlp.get_pipe("tagger").labels if "Number=Plur" in l]))

ADJ__Gender=Fem|Number=Plur
ADJ__Gender=Fem|Number=Plur|NumType=Ord
ADJ__Gender=Masc|Number=Plur
ADJ__Gender=Masc|Number=Plur|NumType=Ord
ADJ__Gender=Unsp|Number=Plur
ADJ__Number=Plur
ADP_DET__Definite=Def|Gender=Fem|Number=Plur|PronType=Art
ADP_DET__Definite=Def|Gender=Masc|Number=Plur|PronType=Art
ADP_DET__Gender=Fem|Number=Plur|PronType=Art
ADP_DET__Gender=Fem|Number=Plur|PronType=Dem
ADP_DET__Gender=Fem|Number=Plur|PronType=Ind
ADP_DET__Gender=Masc|Number=Plur|PronType=Art
ADP_DET__Gender=Masc|Number=Plur|PronType=Dem
ADP_DET__Gender=Masc|Number=Plur|PronType=Ind
ADP_PRON__Case=Nom|Gender=Fem|Number=Plur|Person=3|PronType=Prs
ADP_PRON__Case=Nom|Gender=Masc|Number=Plur|Person=3|PronType=Prs
ADP_PRON__Definite=Def|Gender=Fem|Number=Plur|PronType=Art
ADP_PRON__Definite=Def|Gender=Masc|Number=Plur|PronType=Dem
ADP_PRON__Gender=Fem|Number=Plur|Person=1|PronType=Prs
ADP_PRON__Gender=Fem|Number=Plur|Person=3|PronType=Prs
ADP_PRON__Gender=Fem|Number=Plur|PronType=Dem
ADP_PRON__Gender=Fem|N